In [31]:
import scipy.stats as ss, pandas as pd, numpy as np

from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [24]:
y_train = pd.read_csv('Y_train.csv', index_col=0)
y_valid = pd.read_csv('Y_valid.csv', index_col=0)
y = np.asarray(pd.concat([y_train, y_valid])).flatten()

x_train = pd.read_csv('X_train_cc.csv', index_col=0)
x_valid = pd.read_csv('X_valid_cc.csv', index_col=0)
X = pd.concat([x_train, x_valid])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

In [28]:
xgb1 = XGBRegressor()
parameters = {'nthread':[4],
              'objective':['reg:squarederror'],
              'learning_rate': [0.01, 0.1],
              'max_depth': [5, 10],
              'min_child_weight': [4, 6],
              'subsample': [0.7],
              'colsample_bytree': [0.7],
              'n_estimators': [500]}

xgb_grid = GridSearchCV(xgb1, parameters, cv = 2, n_jobs = 5, verbose=True)
xgb_grid.fit(X_train, y_train)
print(xgb_grid.best_score_)

Fitting 2 folds for each of 8 candidates, totalling 16 fits
0.1905487342630326


In [29]:
xgb2 = XGBRegressor(**xgb_grid.best_params_)
xgb2.fit(X_train, y_train)
predictions = xgb2.predict(X_test)

In [37]:
print("============== score: ", np.round(xgb2.score(X_test, y_test), 5), "==============")
print("============== MSE:   ", np.round(mean_squared_error(y_test, xgb2.predict(X_test)), 5), "==============")

pd.DataFrame({'Id': X_test.index, 'Predicted': predictions}).set_index('Id')


============== score:  0.18715 ==============
============== MSE:    3.13015 ==============


,Predicted
Id,
12732_142290_292064,4.054928
4273_158748_243237,3.431731
24979_108776_280967,4.216048
17276_165268_217606,2.726931
24235_155616_203101,2.590534
...,...
25662_198249_241244,3.227424
47667_140010_273449,2.517060
32605_157470_217181,5.513839
